In [401]:
global_namespace = Dict()

function metajulia_repl()
    print(">>", " ")
    ans = readline()
    parsed_ans = Meta.parse(ans)
    my_eval(parsed_ans, global_namespace)
end

function my_eval(expr, namespace)
    if !(expr isa Expr)
        if expr isa Symbol
            return namespace[expr]
        else
            return expr
        end
    else
        if expr.head == :block
            for arg in expr.args[begin:end-1]
                if !(arg isa LineNumberNode)
                    my_eval(arg, namespace)
                end
            end
            return !(expr.args[end] isa LineNumberNode) ? my_eval(expr.args[end], namespace) : nothing
        elseif expr.head == :&&
            return reduce(&, [my_eval(expr.args[x], namespace) for x in 1:length(expr.args)])
        elseif expr.head == :||
            return reduce(|, [my_eval(expr.args[x], namespace) for x in 1:length(expr.args)])
        elseif expr.head == :if
            return my_eval(expr.args[1], namespace) ? my_eval(expr.args[2], namespace) : my_eval(expr.args[3], namespace)
        elseif expr.head == :elseif
            return my_eval(expr.args[1].args[2]) ? my_eval(expr.args[2], namespace) : my_eval(expr.args[3], namespace)
        elseif expr.head == :(=)
            namespace[expr.args[1]] = my_eval(expr.args[2], namespace)
        elseif expr.head == :let
            # println("let")
            # println(expr.args[begin])
            # println(expr.args[end])
            let
                scope_dict = copy(namespace)
                for arg in expr.args[begin:end-1]
                    if !(arg isa LineNumberNode)
                        my_eval(arg, scope_dict)
                    end
                end
                # println(!(expr.args[end] isa LineNumberNode))
                return !(expr.args[end] isa LineNumberNode) ? my_eval(expr.args[end], scope_dict) : nothing
            end
        elseif expr.head == :call
            if expr.args[1] == :+
                return reduce(+, [my_eval(expr.args[x], namespace) for x in 2:length(expr.args)])
            elseif expr.args[1] == :*
                return reduce(*, [my_eval(expr.args[x], namespace) for x in 2:length(expr.args)])
            elseif expr.args[1] == :/
                return reduce(/, [my_eval(expr.args[x], namespace) for x in 2:length(expr.args)])
            elseif expr.args[1] == :<
                return reduce(<, [my_eval(expr.args[x], namespace) for x in 2:length(expr.args)])
            elseif expr.args[1] == :>
                return reduce(>, [my_eval(expr.args[x], namespace) for x in 2:length(expr.args)])
            end
        end
    end
end

my_eval (generic function with 2 methods)

In [409]:
metajulia_repl()

>> 

stdin>  b


LoadError: KeyError: key :b not found

In [283]:
Meta.parse("x = 1").args[1] isa Symbol

true

In [246]:
my_eval()

LoadError: MethodError: no method matching my_eval()

[0mClosest candidates are:
[0m  my_eval([91m::Any[39m)
[0m[90m   @[39m [35mMain[39m [90m[4mIn[242]:9[24m[39m


In [211]:
Meta.parse("let x = 1; y = 2; z = 3; x end").args


2-element Vector{Any}:
 :(x = 1)
 quote
    #= none:1 =#
    y = 2
    #= none:1 =#
    z = 3
    #= none:1 =#
    x
end

In [333]:
Meta.parse("let var = 100 end").args[end].args

1-element Vector{Any}:
 :(#= none:1 =#)

In [227]:
x = 1

1

In [228]:
typeof(x)

Int64